In [125]:
# load model from hf_cache
import os
from pathlib import Path

cache_dir = Path.cwd() / "hf_cache"
cache_dir.mkdir(exist_ok=True)
os.environ["HF_HOME"] = str(cache_dir)

In [126]:
cache_dir

PosixPath('/n/netscratch/sham_lab/Lab/jlsun/InstructedToBias/hf_cache')

In [3]:
import openai
import json
import numpy as np
import random

from transformers import AutoTokenizer, AutoModelForCausalLM
from baukit import Trace
import torch
from copy import deepcopy
from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device: {device}")

device: cuda


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# set dpi to 300
plt.rcParams['figure.dpi'] = 300

In [5]:
from dotenv import load_dotenv
load_dotenv()

False

In [6]:
MODEL_NAME = "allenai/Olmo-3-7B-Instruct"

In [66]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto", cache_dir=cache_dir, low_cpu_mem_usage=True, torch_dtype=torch.bfloat16).eval()
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [135]:
df = pd.read_csv("Question_Subsets/certainty_effect_pairs.csv", index_col=0).reset_index()

In [137]:
RANDOM_STATE = 42
df_sample = df.sample(10, random_state=RANDOM_STATE)

In [138]:
def get_message(prompt, system_prompt=None):
    if system_prompt:
        return [{"role": "system", "content": system_prompt}, {"role": "user", "content": prompt}]
    else:
        return [{"role": "user", "content": prompt}]

In [139]:
system_prompt = "You are a helpful assistant. Answer shortly with only your choice with no explanation."

In [140]:
def get_response(input):
    msg = get_message(input, system_prompt)
    input = tokenizer.apply_chat_template(msg, return_tensors="pt", add_generation_prompt=True, return_dict=True).to(device)
    gen = model.generate(**input, max_new_tokens=200, pad_token_id=tokenizer.eos_token_id, do_sample=False)
    return tokenizer.decode(gen[0][len(input.input_ids[0]):], skip_special_tokens=True)

In [141]:
# test pair
i = 3

control_text = df_sample.iloc[i]['control_text']
treatment_text = df_sample.iloc[i]['treatment_text']

In [142]:
print(control_text)

Below you will find two options for getting a prize.
Option A - $1000 with a 25% chance, $0 with a 75% chance.
Option B - $850 with a 26% chance, $0 with a 74% chance.
Given that these are the only options, which one would you choose?
Answer:


In [143]:
print(treatment_text)

Below you will find two options for getting a prize.
Option A - $1000 with a 25% chance, $850 with a 74% chance, $0 with a 1% chance.
Option B - $850 with certainty.
Given that these are the only options, which one would you choose?
Answer:


In [144]:
df_sample.iloc[i]

index                                                              144
match_key                                           (2, 1, 5, 1, 1, 1)
control_idx                                                        144
treatment_idx                                                      216
control_text         Below you will find two options for getting a ...
treatment_text       Below you will find two options for getting a ...
template                                                             2
bias_type_index                                                      1
vals_index                                                           5
permutation_index                                                    1
treatment_answer                                                target
control_answer                                   better_expected_value
treatment_prob                                               -0.632625
control_prob                                                 -0.252373
target

In [145]:
for i in range(len(df)):
    control_text = df.iloc[i]['control_text']
    treatment_text = df.iloc[i]['treatment_text']

    control_resp = get_response(control_text)
    treatment_resp = get_response(treatment_text)
    print(i)
    print(control_resp)
    print(treatment_resp)
    print('---')

0
A
Option B
---
1
Option A
A
---
2
Option A
Option A
---
3
A
A
---
4
Option B
A
---
5
A
A
---
6
Option A
Option A
---
7
A
A
---
8
Option B
A
---
9
Option A
A
---
10
A
A
---
11
A
Option B
---
12
B
A
---
13
A
A
---
14
A
Option B
---
15
B
A
---
16
A
A
---
17
B
A
---
18
A
A
---
19
A
B
---
20
A
B
---
21
B
A
---
22
A
A
---
23
B
A
---
24
Option B
A
---
25
A
Option B
---
26
B
A
---
27
Option A
B
---
28
A
A
---
29
B
A
---
30
A
Option B
---
31
Option B
A
---
32
Option B
A
---
33
A
A
---
34
B
A
---
35
A
A
---
36
Option A
B
---
37
Option B
B
---
38
A
Option B
---
39
B
A
---
40
Option A
B
---
41
Option B
A
---
42
A
A
---
43
Option A
B
---
44
A
B
---
45
A
A
---
46
B
A
---
47
A
B
---
48
A
B
---
49
A
Option B
---
50
A
B
---
51
A
B
---
52
B
A
---
53
A
B
---
54
A
B
---
55
A
A
---
56
B
A
---
57
A
B
---
58
A
B
---
59
B
A
---
60
A
Option B
---
61
B
A
---
62
A
B
---
63
A
B
---
64
B
A
---
65
B
A
---
66
A
B
---
67
A
B
---
68
B
A
---
69
A
B
---
70
A
B
---
71
B
A
---
72
A
B
---
73
A
B
---
74
B
A
---
75
A
B
---